<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/L03_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# QSD Lab 3 Tests
# Copyright 2024 The MITRE Corporation. All Rights Reserved.

!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [3]:
# Generate random rotations and apply them to qubits

import numpy as np

def generate_random_rotation():
    return [
        np.random.uniform(0, np.pi),
        np.random.uniform(0, 2 * np.pi)
    ]

def apply_rotation(rotation, target, qc):
    qc.rx(rotation[0], target)
    qc.rz(rotation[1], target)


In [ ]:
# Test E01

def E01Test():
    for i in range(5):
        qr = QuantumRegister(2)
        qc = QuantumCircuit(qr)

        rotations = [generate_random_rotation(), generate_random_rotation()]

        apply_rotation(rotations[0], qr[0], qc)
        apply_rotation(rotations[1], qr[1], qc)

        qc.swap(qr[0], qr[1])

        apply_rotation(rotations[0], qr[1], qc.inverse())
        apply_rotation(rotations[1], qr[0], qc.inverse())

        qc.measure_all()

        t_qc = transpile(qc, sim)
        qobj = assemble(t_qc)
        result = sim.run(qobj).result()
        counts = result.get_counts()

        print("\nCounts for E01Test iteration {}:".format(i+1), counts)

E01Test()

In [ ]:
# Test E02

def E02_ReverseRegister(register_length):
    qr = QuantumRegister(register_length, 'q')
    qc = QuantumCircuit(qr)

    for i in range(register_length // 2):
        qc.swap(qr[i], qr[register_length - 1 - i])

    return qc

def E02Test():
    for numQubits in range(3, 9):
        qr = QuantumRegister(numQubits)
        qc = QuantumCircuit(qr)

        rotations = []
        for index in range(numQubits):
            rotation = generate_random_rotation()
            rotations.append(rotation)
            apply_rotation(rotation, qr[index], qc)

        reverse_qc = E02_ReverseRegister(numQubits)
        qc = qc.compose(reverse_qc)

        for index in range(numQubits):
            apply_rotation(rotations[numQubits - index - 1], qr[index], qc.inverse())

        qc.measure_all()

        t_qc = transpile(qc, sim)
        qobj = assemble(t_qc)
        result = sim.run(qobj).result()
        counts = result.get_counts()

        print("\nCounts for E02Test with {} qubits:".format(numQubits), counts)

E02Test()

In [ ]:
# Test E03
def E03_PrepareBellStates():
    registers = [QuantumRegister(2, f'reg{i}') for i in range(4)]
    qc = QuantumCircuit(*registers)

    for i, reg in enumerate(registers):
        qc.h(reg[0])
        qc.cx(reg[0], reg[1])

    qc.z(registers[1][0])
    qc.x(registers[2][1])
    qc.z(registers[3][0])
    qc.x(registers[3][1])

    return qc

def E03Test():
    qr = QuantumRegister(8, 'q')
    qc = QuantumCircuit(qr)
    registers = [
        [qr[0], qr[1]],
        [qr[2], qr[3]],
        [qr[4], qr[5]],
        [qr[6], qr[7]]
    ]

    bell_circuit = E03_PrepareBellStates()
    qc = qc.compose(bell_circuit)

    for reg in registers:
        qc.cx(reg[0], reg[1])
        qc.h(reg[0])

    qc.x(qr[2])
    qc.x(qr[5])
    qc.x(qr[6])
    qc.x(qr[7])

    qc.measure_all()

    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("\nCounts for E03Test:", counts)

E03Test()

In [ ]:
# Test E04

def E04Test():
    for numQubits in range(2, 11):
        qr = QuantumRegister(numQubits)
        qc = QuantumCircuit(qr)

        qc.h(qr[0])
        for index in range(1, numQubits):
            qc.cx(qr[0], qr[index])

        for index in range(1, numQubits):
            qc.cx(qr[0], qr[index])
        qc.h(qr[0])

        qc.measure_all()

        t_qc = transpile(qc, sim)
        qobj = assemble(t_qc)
        result = sim.run(qobj).result()
        counts = result.get_counts()

        print("\nCounts for E04Test with {} qubits:".format(numQubits), counts)

E04Test()

In [ ]:
# Test E05
def E05Test():
    qr = QuantumRegister(4)
    qc = QuantumCircuit(qr)

    # Apply the gates as specified
    qc.z(qr[2])
    qc.x(qr[3])
    qc.cx(qr[2], qr[3])
    qc.h(qr[2])
    qc.x(qr[1])

    qc.measure_all()

    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("\nCounts for E05Test:", counts)

E05Test()

In [ ]:
# Test E06
def E06Test():
    qr = QuantumRegister(2)
    qc = QuantumCircuit(qr)

    qc.h(qr[0])
    qc.cry(2 * np.arcsin(1/2), qr[0], qr[1])
    qc.measure_all()

    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("\nCounts for E06Test:", counts)

E06Test()

In [ ]:
# Test E07

def E07_EntangleTarget():
    qr = QuantumRegister(3, 'q')
    target = QuantumRegister(1, 'target')
    qc = QuantumCircuit(qr, target)

    for qubit in qr:
        qc.h(qubit)

    qc.x(target[0])
    qc.cx(qr[2], target[0])
    qc.cx(qr[1], target[0])
    qc.cx(qr[0], target[0])

    return qc

def E07Test():
    qr = QuantumRegister(3, 'q')
    target = QuantumRegister(1, 'target')
    qc = QuantumCircuit(qr, target)

    entangle_circuit = E07_EntangleTarget()
    qc = qc.compose(entangle_circuit)

    qc.x(qr[0])
    qc.x(qr[1])
    qc.cx(qr[0], target[0])
    qc.cx(qr[1], target[0])
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])

    qc.measure_all()

    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("\nCounts for E07Test:", counts)

E07Test()

In [ ]:
# Test E08

def E08_PrepareComplexState(qc, qr):
    # Apply the operations to prepare the complex state
    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.cx(qr[1], qr[2])
    qc.z(qr[2])

    return qc

def E08Test():
    qr = QuantumRegister(3, 'q')
    ancilla = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(qr, ancilla)

    qc = E08_PrepareComplexState(qc, qr)

    qc.x(qr[1])
    qc.x(qr[2])

    # Ancilla qubit manipulation
    qc.x(ancilla[0])
    qc.cz(qr, ancilla[0])
    qc.x(ancilla[0])

    qc.x(qr[1])
    qc.x(qr[2])
    qc.cx(qr[1], qr[2])
    qc.ch(qr[0], qr[1])
    qc.h(qr[0])

    qc.measure_all()

    t_qc = transpile(qc, sim)
    qobj = assemble(t_qc)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("\nCounts for E08Test:", counts)

E08Test()